In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/All Graphs/final_processed_data/final_processed_april10.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
data

,user_id,text_duplicate,final
0,2.879999e+09,@MeetThePress @chucktodd If Trump is doing suc...,trump great job the usa 30 the global covid19 ...
1,1.631563e+09,"@MyCielola, @FIOB_Oficial, @MayanLeague among ...",among indigenous organization denounce discrim...
2,4.175449e+08,It’s spring break and I’m supposed to be visit...,spring break suppose to visit family tuscaloo...
3,1.003550e+09,One of my hardworking pharmacy techs fell ill ...,one hardworking pharmacy tech fell ill today c...
4,1.238635e+18,Looking for a quarantine boyfriend. Just text ...,look quarantine boyfriend text good morning go...
...,...,...,...
12993,3.821768e+08,Tonight’s virtual town hall with the RNS COVID...,tonight virtual town hall the rn covid19 task...
12994,1.230608e+18,ICYMI: Commissioner @HodgenMainda is featured ...,icymi commissioner feature the new issue the t...
12995,1.238883e+18,COVID Insurance Update 4/9/2020 https://t.co/j...,covid insurance update 492020 via
12996,1.039776e+08,Good thing there are no rush hours these days....,good thing no rush hour day sq issue 1546 fine...


In [4]:
keywords = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

##### 1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [5]:
#Convert to lower
for i in range(len(keywords)): 
    keywords[i] = keywords[i].lower()

#Remove punctuations   
for i in range(len(keywords)): 
    keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords)): 
    keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_tokens = [sub.split() for sub in keywords] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_filtered=remove_stopwords(keywords)

#Stemming
ps = PorterStemmer()
keywords_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_filtered]
keywords_stem = [" ".join(sentence) for sentence in keywords_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_filtered]
keywords_final = [" ".join(sentence) for sentence in keywords_lem]

## Fuzzy Matching

In [6]:
data['final'] = data['final'].apply(str)
choices = data['final'].tolist()

In [7]:
def fuzzy_m(row):
        keyword_match, score = process.extractOne(row['final'], keywords_final, scorer = fuzz.partial_ratio)
        row['final_score'] = score
        row['final_keyword_match'] = keyword_match
        return row

In [8]:
nan_value = float("NaN")
data['final'].replace("", nan_value, inplace=True)
data.dropna(subset = ["final"], inplace=True)

In [10]:
interim_imp = data.apply(fuzzy_m, axis=1)

In [11]:
interim_imp

,user_id,text_duplicate,final,final_score,final_keyword_match
0,2.879999e+09,@MeetThePress @chucktodd If Trump is doing suc...,trump great job the usa 30 the global covid19 ...,67,isolat
1,1.631563e+09,"@MyCielola, @FIOB_Oficial, @MayanLeague among ...",among indigenous organization denounce discrim...,64,responsible
2,4.175449e+08,It’s spring break and I’m supposed to be visit...,spring break suppose to visit family tuscaloo...,100,house
3,1.003550e+09,One of my hardworking pharmacy techs fell ill ...,one hardworking pharmacy tech fell ill today c...,75,stay at home
4,1.238635e+18,Looking for a quarantine boyfriend. Just text ...,look quarantine boyfriend text good morning go...,100,quarantine
...,...,...,...,...,...
12993,3.821768e+08,Tonight’s virtual town hall with the RNS COVID...,tonight virtual town hall the rn covid19 task...,50,stay at home
12994,1.230608e+18,ICYMI: Commissioner @HodgenMainda is featured ...,icymi commissioner feature the new issue the t...,67,cancel
12995,1.238883e+18,COVID Insurance Update 4/9/2020 https://t.co/j...,covid insurance update 492020 via,67,cancel
12996,1.039776e+08,Good thing there are no rush hours these days....,good thing no rush hour day sq issue 1546 fine...,57,closure


In [12]:
denominator = interim_imp.shape[0]

In [13]:
interim_imp['final_score'] = interim_imp['final_score'].astype(int)

In [14]:
interim_imp = interim_imp[interim_imp['final_score'] == 100]

In [15]:
interim_imp

,user_id,text_duplicate,final,final_score,final_keyword_match
2,4.175449e+08,It’s spring break and I’m supposed to be visit...,spring break suppose to visit family tuscaloo...,100,house
4,1.238635e+18,Looking for a quarantine boyfriend. Just text ...,look quarantine boyfriend text good morning go...,100,quarantine
7,2.849173e+09,Ima need a Covid therapist after this quaranti...,ima need covid therapist quarantine no capppppp,100,quarantine
24,1.229759e+18,@TimNoEgo @FrakerMonica What Trump means is th...,trump mean grueling to stretch out watch quara...,100,quarantine
26,2.146648e+07,What a great idea! I’ll be watching. #Passover...,great idea watch passover2020 shelterinplace ...,100,home
...,...,...,...,...,...
12887,1.598221e+08,I'm taking advantage of recent event cancellat...,im take advantage recent event cancellation le...,100,cancel
12966,2.992390e+08,"Criminals don’t obey gun laws, so what makes a...",criminal obey gun law make anyone think obey ...,100,stay at home
12971,7.631975e+17,Me in the club when this quarantine is over ht...,the club quarantine,100,quarantine
12979,7.855487e+08,"This week, MHOF connected art coordinators acr...",week mhof connect art coordinator across the n...,100,school closure


In [16]:
numerator = interim_imp.shape[0]

In [17]:
proportion = (numerator/denominator)
print(proportion)

0.1301738729035236
